### --------------------------------------------------------------------------------------------------
## 社会変革型 医療データサイエンティスト育成講座
# Chapter 2: ロジスティック回帰モデル
### --------------------------------------------------------------------------------------------------

#### 1. sklearnを用いたロジスティック回帰モデル構築

In [ ]:
# データのロード
import pandas as pd

# ホームディレクトリにデータを格納していれば、フォルダの指定は必要ありません
bace_data = pd.read_csv('~/bace_data.csv')
bace_data.head()

In [ ]:
# 判別モデルを構築するため、目的変数として'class'（結合=1、非結合=0の二値が入っている）を使用します
# 説明変数としては、'RB'(rotable bond)および'HeavyAtomCount'を使用してみます

import numpy as np
X = np.array(bace_data[['RB','HeavyAtomCount']])
y = np.array(bace_data[['class']])

In [ ]:
# scikit-learnによるモデル構築


# インスタンスの作成


# データのフィッティング


# 回帰係数の出力
print('偏回帰係数 (RB): ', model.coef_[0][0])
print('偏回帰係数 (HeavyAtom): ', model.coef_[0][1])
print('切片: ', model.intercept_[0])

In [ ]:
# (参考) 結果の可視化
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

# 散布図のための関数定義
def plot_scatter(df_pos,df_neg,cols=['RB','HeavyAtomCount'],dsize=10):
    plt.scatter(df_pos[cols[0]],df_pos[cols[1]],
                c='orange',
                s=dsize,
                label='positive',
               )
    plt.scatter(df_neg[cols[0]],df_neg[cols[1]],
                c='blue',
                s=dsize,
                label='negative',
                alpha=0.3, # そのままだとドットが重なって見にくいため、透明度を調整
               )

    # legendを可視化
    plt.legend()

# 変数定義
data_rb = bace_data['RB']
data_ha = bace_data['HeavyAtomCount']
df_pos = bace_data[bace_data['class']==1]
df_neg = bace_data[bace_data['class']==0]

# プロットのための座標を作成します
rb = np.linspace(np.min(data_rb),np.max(data_rb),51)
ha = np.linspace(np.min(data_ha),np.max(data_ha),51)

# age x BMI の２次元平面の各座標において、regressionで求めた関数から陽性率を計算します
prediction = np.zeros([51,51])
for i in range(51):
    for j in range(51):
        prediction[i,j] = model.predict_proba([[rb[i],ha[j]]])[0,1]

# 陽性率の関数とデータの散布図を同一グラフ上に描画
extent = [np.min(data_rb), np.max(data_rb), np.min(data_ha), np.max(data_ha)]
fig = plt.figure(figsize=[9,6])
ax = fig.add_subplot(1,1,1)
plot_scatter(df_pos,df_neg,dsize=15)
im = ax.imshow(prediction.T,
               extent=extent,
               origin='lower',
               cmap='pink'
              )
ax.set_xlabel('RB')
ax.set_ylabel('HeavyAtomCount')
ax.set_ylim([10,60])
fig.colorbar(im)
plt.show()

#### おまけ. シンプソンパラドックス

In [ ]:
# 関数定義
def create_df(n, rate, labels):
    df = pd.DataFrame(data={'性別':[labels[0] for i in range(n)], 
                         '受験学部':[labels[1] for i in range(n)],
                         '合否':[1 for i in range(np.int(np.round(n*rate)))] + [0 for i in range(np.int(np.round(n*(1-rate))))]})
    return df

In [ ]:
# データを作成（ウェブ上で見つけられなかったため、数値を入れつつ自作する）

w_A = create_df(108,0.82,['w','A'])
w_B = create_df(25,0.68,['w','B'])
w_C = create_df(593,0.34,['w','C'])
w_D = create_df(375,0.35,['w','D'])
w_E = create_df(393,0.24,['w','E'])
w_F = create_df(341,0.07,['w','F'])

m_A = create_df(825,0.619,['m','A'])
m_B = create_df(560,0.63,['m','B'])
m_C = create_df(325,0.37,['m','C'])
m_D = create_df(417,0.33,['m','D'])
m_E = create_df(191,0.28,['m','E'])
m_F = create_df(272,0.06,['m','F'])

dataset = pd.concat([w_A,w_B,w_C,w_D,w_E,w_F,m_A,m_B,m_C,m_D,m_E,m_F],ignore_index=True).reset_index(drop=True)

In [ ]:
# データセットの行をシャッフルし、中身を確認
dataset.sample(frac=1).reset_index(drop=True).head(10)

In [ ]:
# 全体の合格率
w = dataset[dataset['性別']=='w']
m = dataset[dataset['性別']=='m']
print('女性の人数は%s人で、合格率は%sパーセント' % (len(w), np.round(100*len(w[w['合否']==1])/len(w),1)))
print('男性の人数は%s人で、合格率は%sパーセント' % (len(m), np.round(100*len(m[m['合否']==1])/len(m),1)))

In [ ]:
# 性別および受験学部をダミー変数に変換
dataset_dum = pd.get_dummies(dataset,prefix=['性別','受験学部'], drop_first=True)

# 性別は女性が1、男性が0
# 受験学部は、B~Fがそれぞれのカラムでラベルされており、全て0であればAを表す
dataset_dum.head()

In [ ]:
# scikit-learnによるモデル構築
from sklearn.linear_model import LogisticRegression

# インスタンスの作成
logisticModel = LogisticRegression()

# データのフィッティング
logisticModel.fit(dataset_dum.iloc[:,1:],dataset_dum['合否'])

# 回帰係数の出力
print('男性に対する女性のオッズ比: ',np.exp(logisticModel.coef_[0][0]))
print('その他の係数: ', logisticModel.coef_[0][1:])

・　「受験学部」という交絡因子の影響を除いた場合、女性の方が合格率がわずかに高いという結果になった。  
・　また、その他の係数が全てマイナスであることから、学部Aへの合格率が最も高く、学部Fが最も低いことがわかる。